In [41]:
import numpy as np
import random
import math

In [70]:
LIMITS = 100
DIMENSION = 5
MAX_FITNESS = 1
MAX_ITER = 100
SAMPLE_SIZE = 100
INITIAL_SAMPLE_SIZE = 50

In [43]:
class Solution:
    def __init__(self, sol, f = 0):
        self.sol = sol
        self.f = f

    def set_f(self, f):
        self.f = f

In [44]:
def esfera(sol):
    sum = 0
    for xi in sol:
        sum += xi ** 2
    return sum

In [45]:
def fitness(sol):
    maxi = DIMENSION * LIMITS * LIMITS
    assert(maxi > 0)
    esf = esfera(sol)
    assert(esf <= maxi)
    return (maxi - esf) / maxi

In [46]:
def f(sol):
    fit = fitness(sol.sol)
    sol.set_f(fit)
    #print(f"FITNESS de {sol.sol} = {fit}")
    return fit

In [47]:
def manhattan_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return sum([abs(sol1[i] - sol2[i]) for i in range(n)])

In [48]:
def euclidean_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return (math.sqrt(sum([(sol1[i] - sol2[i]) ** 2 for i in range(n)])))

In [49]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False)

In [50]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    assert(d > 0) # Creo que todo está preparado para no samplear dos iguales.
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [51]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.25

In [52]:
def search_nearest(sol, A, dist):
    card = len(A)
    assert(card > 0)
    i_nearest = 0
    min_distance = dist(sol.sol, A[i_nearest].sol)
    for i in range(1, card):
        distance = dist(sol.sol, A[i].sol)
        if distance < min_distance:
            min_distance = distance
            i_nearest = i
    return A[i_nearest], i_nearest

In [53]:
def search_optimal(A):
    assert (len(A) > 0)
    return A[0]

In [54]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for point in A:
        sol = point.sol
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [55]:
def g(sol, A, dist):
    sum = 0
    k = len(A)
    for i in range(k):
        wei = w(sol.sol, A[i].sol, A, dist)
        fit = (k - i) / k
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    #print(f"\n\n g: {sum}\n\n")
    return sum

In [56]:
def g_positional(sol, set_sols, i_nearest, size, dist):
    wei_opt = w(sol.sol, set_sols[0].sol, set_sols, dist)
    fit_opt = 1
    wei_nearest = w(sol.sol, set_sols[1].sol, set_sols, dist)
    fit_nearest = (size - i_nearest) / size
    return wei_opt * fit_opt + wei_nearest * fit_nearest

In [57]:
def sample(k, A, dist, skip = []):
    n = DIMENSION
    sample = []
    while len(sample) < k:
        x = [random.randint(-LIMITS, LIMITS) for _ in range(n)]
        if any(point.sol == x for point in sample) or any(point.sol == x for point in skip):
            continue
        #y = random.randint(0, MAX_FITNESS)
        #y = random.uniform(0, MAX_FITNESS)
        # Para fitness al cuadrado:
        y = random.uniform(0, MAX_FITNESS)
        sol = Solution(x)
        opt = search_optimal(A)
        nearest, i_nearest = search_nearest(sol, A, dist)
        set_points = [opt, nearest]
        g_val = g_positional(sol, set_points, i_nearest, k, dist)
        # Para fitness al cuadrado:
        #g_val = g_val ** 30
        if g_val > y:
            #print(fitness(x))
            #print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            f(sol)
            sample.append(sol)
    return sample

In [58]:
def init(k):
    n = DIMENSION
    init = []
    while len(init) < k:
        x = [random.randint(-LIMITS, LIMITS) for _ in range(n)]
        if not any(point.sol == x for point in init): # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            sol = Solution(x)
            f(sol)
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [59]:
def max_f(A):
    assert(len(A) > 0)
    max_value = A[0].f
    max_elem = A[0]
    for elem in A[1:]:
        fit = elem.f
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [60]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    #dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [61]:
def next_swarm_step(k, A, dist):
    opt = search_optimal(A)
    next = sample(k - 1, A, dist, skip = A)
    next.append(opt)
    return sorted(next, key = lambda x : x.f, reverse = True)

In [68]:
def alg2(sample_size, num_iter, dist):
    A = init(sample_size)
    for i in range(num_iter):
        #print(f"iter {i}")
        A = next_swarm_step(sample_size, A, dist)
    return max_f(A)

In [71]:
elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = euclidean_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con volumen {esfera(elem.sol)}.')

Máximo valor de fitness (0.99504) encontrado para [-7, 7, -11, 5, -2], con volumen 248.


In [72]:
for i in range(20):
    elem, fit = alg2(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, dist = euclidean_distance)
    print(f'Máximo valor de fitness ({fit}) encontrado para {elem.sol}, con volumen {esfera(elem.sol)}.')

Máximo valor de fitness (0.99224) encontrado para [-11, -13, -3, 5, 8], con volumen 388.
Máximo valor de fitness (0.99734) encontrado para [8, 4, -2, -7, 0], con volumen 133.
Máximo valor de fitness (0.99658) encontrado para [-5, 0, 1, -9, 8], con volumen 171.
Máximo valor de fitness (0.99492) encontrado para [9, 6, -10, 1, -6], con volumen 254.
Máximo valor de fitness (0.989) encontrado para [10, 15, -14, 2, -5], con volumen 550.
Máximo valor de fitness (0.99476) encontrado para [0, 8, -5, 2, -13], con volumen 262.
Máximo valor de fitness (0.99254) encontrado para [11, 4, -6, 10, 10], con volumen 373.
Máximo valor de fitness (0.99316) encontrado para [-13, -4, 12, -2, 3], con volumen 342.
Máximo valor de fitness (0.99812) encontrado para [-4, -7, 4, -2, 3], con volumen 94.
Máximo valor de fitness (0.99848) encontrado para [1, -3, 4, 5, 5], con volumen 76.
Máximo valor de fitness (0.99252) encontrado para [-4, 2, -7, -16, 7], con volumen 374.
Máximo valor de fitness (0.9958) encontrado